In [2]:
import os

In [3]:
import json
pwc_data = [json.loads(line) for line in open('../data/pwc_s2_cleaned_text_v2.jsonl')]

In [ ]:
scibert_data_path = '/net/nfs.corp/s2-research/scibert/raw-data/'
files = os.listdir(scibert_data_path)
files = [f for f in files if '3800' not in f]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [5]:
tfidf_vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(3, 3), min_df=10, dtype=np.float32)

In [6]:
from tqdm import tqdm
def return_document(files) :
    for f in tqdm(files) :
        text = open(scibert_data_path + f).read()
        text = text.split('\n\n')
        for s in tqdm(text) :
            yield s
            
def return_pwc_document(pwc_data) :
    for d in tqdm(pwc_data) :
        yield d['cleaned_text']

In [7]:
tfidf_vectorizer.fit(return_pwc_document(pwc_data))

100%|██████████| 7844/7844 [01:09<00:00, 94.41it/s] 


TfidfVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
                dtype=<class 'numpy.float32'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=10, ngram_range=(3, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [13]:
b = tfidf_vectorizer.transform(['SQAD', 'SQuAD'])

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
cosine_similarity(b[0], b[1], dense_output=True)

array([[0.18803576]], dtype=float32)